# Homework 3: Unsupervised Learning

# Part 1: Classification with K-means algorithm

The K-means algorithm is a fundamental tool among the unsupervised learning model. Consider a problem with a dataset $\mathcal{D} = \left\{x_i\right\}_{i=1}^n$ where $x_i \in \mathbb{R}^d$ with no labels, we are aiming at finding some hidden structure within the data, namely, we would like to find clusters in the dataset. Classifiers have been studied in TP but mainly for supervised learning, here the data are not labeled. 

The K-means algorithm tries to classify the dataset in $K$ clusters. Each cluster is represented by a centroid, meaning the average of the points within the cluster. We note $C_k$ the set of points of a cluster $k$ and $\mu_k$ its centroid. Then, the algorithm minimizes the intra-cluster variance, in other words, it tries to reduce the distance between the points of the cluster and the centroid. 

More technically, the algorithm works iteratively in two main steps: 
 - Points are assigned to clusters based on their proximity to existing centroid
 - Centroids are updated by taking the average of the points assigned to each cluster.



The Loss function of the problem can be written as:
$$J(\mu_1, ..., \mu_K) = \sum_{i=1}^{n} \, \lVert x_i - \mu(i)\rVert^2 \; ,$$
where $\mu(i)$ is the centroid of the cluster assignated to $x_i$. 

We want to check the understanding of the choice of this loss function. Does it match the aforementioned rules? Then, could it help to understand if the algorithm converges?

Additionnally, until Question 8, we assume that the clusters $C_k$ are disjoint, especially at initialization. 

### The Secret is in the Notation $\mu(i)$
The confusion usually comes from the term $\mu(i)$.
* $\mu_k$ (with a subscript $k$) refers to a specific centroid (e.g., Centroid #1, Centroid #2).
* $\mu(i)$ (function of $i$) is a shorthand way of saying: **"The centroid that point $x_i$ currently belongs to."**

#### A Concrete Example
Imagine you have **3 Data Points** ($x_1, x_2, x_3$) and **2 Clusters** (Centroids $\mu_1, \mu_2$).

* **$x_1$** belongs to Cluster 1.
* **$x_2$** belongs to Cluster 1.
* **$x_3$** belongs to Cluster 2.

The formula $J = \sum_{i=1}^{3} \lVert x_i - \mu(i)\rVert^2$ expands like this:

1.  **For $i=1$:** $x_1$ is in Cluster 1, so $\mu(1)$ is $\mu_1$. Term: $\lVert x_1 - \mu_1 \rVert^2$
2.  **For $i=2$:** $x_2$ is in Cluster 1, so $\mu(2)$ is $\mu_1$. Term: $\lVert x_2 - \mu_1 \rVert^2$
3.  **For $i=3$:** $x_3$ is in Cluster 2, so $\mu(3)$ is $\mu_2$. Term: $\lVert x_3 - \mu_2 \rVert^2$

**Total J:**
$$J = \lVert x_1 - \mu_1 \rVert^2 + \lVert x_2 - \mu_1 \rVert^2 + \lVert x_3 - \mu_2 \rVert^2$$

### Summary
* The sum is over $i$ because we calculate the error **point-by-point**.
* $\mu(i)$ dynamically grabs the correct $\mu_k$ for that specific point.

**Next Step:** This concept is exactly what **Question 1** asks you to prove. Would you like to see how to transform this "point-wise" sum ($\sum_i$) into the "cluster-wise" sum ($\sum_k$)?

#### Question 1:

First, let us familiarize ourselves with the loss function:

 - Prove the second equality:

   $$J(\mu_1, ..., \mu_K) = \sum_{i=1}^{n} \, \lVert x_i - \mu(i)\rVert^2 = \sum_{k=1}^{K} \sum_{x_i \in C_k} \, \lVert x_i - \mu_k \rVert^2$$

 - What does the term $\sum_{x_i \in C_k} \, \lVert x_i - \mu_k \rVert^2$ represent?
 
 - Explain why this form of the loss function is more convenient.


### **Q1 Ans**

To prove the equality, we first define the dataset as $D = \{x_i\}_{i=1}^n$. We assume that we classify the dataset into $K$ disjoint clusters, which means that the union of all clusters $C_1 \cup C_2 \dots \cup C_K$ is equal to the whole dataset $D$, and every point $x_i$ belongs to exactly one specific cluster. If a point $x_i$ is in cluster $C_k$, then its assigned centroid $\mu(i)$ is simply $\mu_k$.

We can rewrite the total loss function by grouping the points based on which cluster they belong to. Instead of summing $i$ from 1 to $n$ in one go, we can split the sum into $K$ parts, calculating the error for each cluster separately:

$$J(\mu_1, ..., \mu_K) = \sum_{x_i \in C_1} \lVert x_i - \mu_1 \rVert^2 + \sum_{x_i \in C_2} \lVert x_i - \mu_2 \rVert^2 + \dots + \sum_{x_i \in C_K} \lVert x_i - \mu_K \rVert^2$$

Since this summation covers every point in the dataset exactly once, we can combine these terms using the summation over $k$. This proves that the two forms are mathematically identical:

$$\sum_{i=1}^{n} \, \lVert x_i - \mu(i)\rVert^2 = \sum_{k=1}^{K} \sum_{x_i \in C_k} \, \lVert x_i - \mu_k \rVert^2$$

For the second question, the term $\sum_{x_i \in C_k} \, \lVert x_i - \mu_k \rVert^2$ represent the sum of the square distance of each points in cluster $k$ to its centroid. Physically, this measures the intra-cluster variance. It tells us how "spread out" the points are within that specific group. A lower value means the cluster is very compact and the points are gathered tightly around the center $\mu_k$.

Finally, this form of the loss function is more convenient because it separates the problem into independent parts. When we want to minimize the loss to find the optimal position for a centroid $\mu_k$, we can ignore all other clusters. The terms for other clusters $C_{j}$ (where $j \neq k$) do not depend on $\mu_k$, so they act as constants. This means we can simplify the optimization problem significantly:

$$\frac{\partial J}{\partial \mu_k} = \frac{\partial}{\partial \mu_k} \left( \sum_{j=1}^K \sum_{x_i \in C_j} \lVert x_i - \mu_j \rVert^2 \right) = \frac{\partial}{\partial \mu_k} \left( \sum_{x_i \in C_k} \lVert x_i - \mu_k \rVert^2 \right)$$

This independence allow us to update each centroid separately without worrying about the rest of the dataset at the same time.


#### Question 2:

Let us focus on the first point of the algorihtm, let us consider a single point $x_i$ and add the time dependency. Also, we denote by a $'$ the variables after the new assignement of the data points. 

So that, the variables are denoted by: $(\cdot)^t \to (\cdot)^t\,{}' \to (\cdot)^{t+1} \to (\cdot)^{t+1}\,{}' \to (\cdot)^{t+2}$


Thus, at each step time $t$, the new assignements of the variables leads to: (no proof required)

$$ \mu^t(i)' = {\rm argmin}_{\mu \in \left\{\mu^t_k\right\}_k} \lVert x_i - \mu \rVert^2$$

For instance, at time $t$ before new assignements, the vector $x_i$ belongs to a cluster $k$, while after the next assignement, it now belongs to the cluster $k'$ (it can be the same or different from the cluster $k$). 


Thus, for the whole dataset, the algorithm updates the assignement as: $\left\{\mu^t(i) \right\} \to \left\{\mu^t(i)'\right\}$.


Compare $\lVert x_i - \mu^t(i) \rVert^2$ and $\lVert x_i - \mu^t(i)' \rVert^2$ for a given point. 

### **Q2 Ans**

To compare the two terms, we first look at the definition of the new assignment provided in the question. The algorithm updates the assignment for a single point $x_i$ by choosing the closest centroid from the current set of centroids:

$$\mu^t(i)' = {\rm argmin}_{\mu \in \left\{\mu^t_k\right\}_k} \lVert x_i - \mu \rVert^2$$

This expression tells us that $\mu^t(i)'$ is the specific centroid that achieves the minimum possible distance to $x_i$ among all available clusters at time $t$. On the other hand, $\mu^t(i)$ represents the centroid that was assigned to the point in the previous step. It is important to note that the old centroid $\mu^t(i)$ belongs to the set of all existing centroids $\left\{\mu^t_k\right\}_k$, which means it was one of the candidates we considered during the minimization process.

Because $\mu^t(i)'$ is selected specifically to be the minimum, the squared distance to it must be less than or equal to the squared distance to any other candidate in the set, including the old assignment $\mu^t(i)$. Therefore, we can conclude the following inequality:

$$\lVert x_i - \mu^t(i)' \rVert^2 \leq \lVert x_i - \mu^t(i) \rVert^2$$

This inequality implies that for every single data point, the reassignment step strictly reduces the distance to the cluster center, or keeps it the same if the point is already at the optimal cluster. The error for a specific point never increases during this step.

#### Question 3:

We recall that we denote by a $'$ the variables after the new assignement of the data points, so that: $\mu^t(i) \to \mu^t(i)'$ and $J_t \to J_t'$

Thanks to the previous question, compare $J_t$ and ${J_t}'$. 

Hint: Pick the right formula between the two given for $J$

### **Q3 Ans**
To compare $J_t$ and $J_t'$, we should use the first formula of the loss function which sums over the index $i$. This is the most convenient choice because in the previous question, we analyzed the behavior of individual data points. We can write the expressions for the loss before and after the assignment as:

$$J_t(\mu_1, \dots, \mu_K) = \sum_{i=1}^{n} \lVert x_i - \mu^t(i)\rVert^2$$

$$J_t'(\mu_1, \dots, \mu_K) = \sum_{i=1}^{n} \lVert x_i - \mu^t(i)'\rVert^2$$

From Question 2, we know that for each data point $x_i$, the squared distance to the new centroid is less than or equal to the squared distance to the old centroid. This is because we specifically chose the closest centroid during the update. So for every $i$, we have the inequality:

$$\lVert x_i - \mu^t(i)' \rVert^2 \leq \lVert x_i - \mu^t(i) \rVert^2$$

Since this inequality hold for every single point in the dataset, we can sum it up from $i=1$ to $n$. If every term on the left side is smaller or equal to the corresponding term on the right side, the total sum must also be smaller or equal:

$$\sum_{i=1}^{n} \lVert x_i - \mu^t(i)' \rVert^2 \leq \sum_{i=1}^{n} \lVert x_i - \mu^t(i) \rVert^2$$

Replacing the sums with the notation for $J$, we can conclude:

$$J_t'(\mu_1, \dots, \mu_K) \leq J_t(\mu_1, \dots, \mu_K)$$

This proves that the assignment step of the algorithm always decrease the total loss function (or keeps it the same), ensuring the algorithm improves the solution.

#### Question 4:

Then, we can go ahead by studying the second point of the algorithm. It wants to minimize the intra-cluster variance:

$$ \left\{\mu^{t+1}_k\right\}_k = {\rm argmin}_{\left\{\mu_k\right\}_k \in \mathbb{R}^d} J'_t\left( \left\{\mu_k\right\}_k \right) = {\rm argmin}_{\left\{\mu_k\right\}_k \in \mathbb{R}^d} \sum_{k=1}^{K} \sum_{x_i \in C^t_k{}'} \, \lVert x_i - \mu_k \rVert^2 $$


Show that the optimization can be done cluster-wise:

$$\mu^{t+1}_k = {\rm argmin}_{\mu_k \in \mathbb{R}^d}  \sum_{x_i \in C^t_k{}'} \, \lVert x_i - \mu_k \rVert^2 $$

### **Q4 Ans**

To show that the optimization can be done cluster-wise, we first expand the objective function $J'_t$ to see its components clearly. The objective is defined as a sum over all clusters $k$ from 1 to $K$:

$$J'_t\left( \left\{\mu_k\right\}_k \right) = \sum_{k=1}^{K} \sum_{x_i \in C^t_k{}'} \lVert x_i - \mu_k \rVert^2$$

We can write this sum explicitly as a series of independent terms:

$$J'_t = \left( \sum_{x_i \in C^t_1{}'} \lVert x_i - \mu_1 \rVert^2 \right) + \dots + \left( \sum_{x_i \in C^t_k{}'} \lVert x_i - \mu_k \rVert^2 \right) + \dots + \left( \sum_{x_i \in C^t_K{}'} \lVert x_i - \mu_K \rVert^2 \right)$$

From the problem description, we know that the clusters $C_k$ are disjoint and the variables $\mu_1, \dots, \mu_K$ are independent. This means that the term for cluster $k$ depends *only* on the variable $\mu_k$ and contains no other centroids. Changing $\mu_k$ has absolutely no effect on the other terms in the sum.

Mathematically, when we want to minimize a sum of independent functions $F(x, y) = f(x) + g(y)$, it is equivalent to minimizing each function separately. Therefore, we can push the "argmin" operator inside the sum and solve for each $\mu_k$ individually:

$${\rm argmin}_{\left\{\mu_k\right\}_k} J'_t = {\rm argmin}_{\mu_1} \left( \dots \right) + \dots + {\rm argmin}_{\mu_k} \left( \sum_{x_i \in C^t_k{}'} \lVert x_i - \mu_k \rVert^2 \right)$$

Thus, the update rule for the specific centroid $k$ becomes:

$$\mu^{t+1}_k = {\rm argmin}_{\mu_k \in \mathbb{R}^d}  \sum_{x_i \in C^t_k{}'} \, \lVert x_i - \mu_k \rVert^2$$

#### Question 5

Show that the new centroids of time $t+1$ are computed according to the following equality:

$$ \mu_k^{t+1} = \frac{1}{|C^t_k{}'|}\sum_{x_i \in C^t_k{}'} x_i $$

Does it correspond to what you expected from the algorithm? 

### **Q5 Ans**

To find the optimal centroid $\mu_k^{t+1}$, we need to minimize the intra-cluster variance. As noted in the handwriting, the objective function for a specific cluster $k$ is a convex quadratic function. The minimum of such a function is obtained where its gradient with respect to $\mu_k$ is equal to zero.

First, let us expand the squared Euclidean norm using vector algebra. We recall that $\lVert v \rVert^2 = v^T v$. So for the objective function, we can write:

$$J(\mu_k) = \sum_{x_i \in C^t_k{}'} \lVert x_i - \mu_k \rVert^2 = \sum_{x_i \in C^t_k{}'} (x_i - \mu_k)^T (x_i - \mu_k)$$

$$= \sum_{x_i \in C^t_k{}'} \left( x_i^T x_i - 2x_i^T \mu_k + \mu_k^T \mu_k \right)$$

Now we compute the gradient $\nabla_{\mu_k}$ of this expression. We treat $x_i$ as a constant. The derivative of $x_i^T x_i$ is 0. The derivative of the linear term $-2x_i^T \mu_k$ is $-2x_i$, and the derivative of the quadratic term $\mu_k^T \mu_k$ is $2\mu_k$. We can apply the gradient operator inside the summation:

$$\nabla_{\mu_k} J(\mu_k) = \sum_{x_i \in C^t_k{}'} \left( 0 - 2x_i + 2\mu_k \right) = \sum_{x_i \in C^t_k{}'} (-2x_i + 2\mu_k)$$

To find the minimum, we set this gradient to zero. We can factor out the constant 2 and split the summation into two distinct parts:

$$-2 \sum_{x_i \in C^t_k{}'} x_i + 2 \sum_{x_i \in C^t_k{}'} \mu_k = 0$$

$$\sum_{x_i \in C^t_k{}'} x_i = \sum_{x_i \in C^t_k{}'} \mu_k$$

In the term on the right, $\mu_k$ does not depend on the index $i$. Therefore, we are simply summing the vector $\mu_k$ repeatedly. The number of times we sum it is equal to the number of points in the cluster. As indicated in the notes, we denote $|C^t_k{}'|$ as the cardinality of the set, which counts how many items are inside $C^t_k{}'$. This allows us to write:

$$\sum_{x_i \in C^t_k{}'} x_i = |C^t_k{}'| \cdot \mu_k$$

$$\mu_k^{t+1} = \frac{1}{|C^t_k{}'|} \sum_{x_i \in C^t_k{}'} x_i$$

Yes, this result corresponds exactly to our expectations. The formula represents the standard mathematical definition of the mean (average). It confirms that to minimize the squared error within a cluster, the representative centroid should be placed at the exact arithmetic average of all the points assigned to that cluster.

#### Question 6:

If we focus on a cluster $k$ at time $t$ after the assignement, noted $C^t_k {}'$, could you compare $\sum_{x_i \in C^t_k{}'} \, \lVert x_i - \mu_k^t \rVert^2$ and $\sum_{x_i \in C^t_k{}'} \, \lVert x_i - \mu_k^{t+1} \rVert^2$ ?

What can you say about ${J_t}'$ and $J_{t+1}$ ? Hint: Use the right formula between the two given for $J$.

### **Q6 Ans**

To compare the two sums, we look back at the definition of the new centroid from Question 4. We defined $\mu_k^{t+1}$ as the specific vector that minimizes the sum of squared distances for the points in the new cluster $C^t_k{}'$. Mathematically, this means it is the global minimum of the function. Because it is the minimum, plugging any other vector into this sum—including the old centroid $\mu_k^t$—must result in a value that is larger or equal. Therefore, for every cluster $k$, we have the following inequality:

$$\sum_{x_i \in C^t_k{}'} \lVert x_i - \mu_k^{t+1} \rVert^2 \leq \sum_{x_i \in C^t_k{}'} \lVert x_i - \mu_k^t \rVert^2$$

Now we can analyze $J_{t'}$ and $J_{t+1}$ using the cluster-wise formula for the loss function. $J_{t'}$ represents the total loss when the points are in their new clusters $C^t_k{}'$ but the centroids are still at their old positions $\mu_k^t$. On the other hand, $J_{t+1}$ is the total loss after we update the centroids to $\mu_k^{t+1}$. We can express them as sums over $k$:

$$J_{t'} = \sum_{k=1}^{K} \sum_{x_i \in C^t_k{}'} \lVert x_i - \mu_k^t \rVert^2$$

$$J_{t+1} = \sum_{k=1}^{K} \sum_{x_i \in C^t_k{}'} \lVert x_i - \mu_k^{t+1} \rVert^2$$

Since we know that the inequality holds for each individual cluster term in the sum, it implies that the sum of all terms must also satisfy the inequality. If every part of a sum gets smaller or stays the same, the total result must also be smaller. Thus, by summing our first inequality from $k=1$ to $K$, we derive the final relation:

$$\sum_{k=1}^{K} \sum_{x_i \in C^t_k{}'} \lVert x_i - \mu_k^{t+1} \rVert^2 \leq \sum_{k=1}^{K} \sum_{x_i \in C^t_k{}'} \lVert x_i - \mu_k^t \rVert^2$$

$$J_{t+1} \leq J_{t'}$$

This confirms that the second step of the algorithm, the update of centroids, always decrease the total loss function.

#### Question 7:

Putting together the Questions 3 and 5, compare $J_t$ and $J_{t+1}$

### **Q7 Ans**
To compare $J_t$ and $J_{t+1}$, we simply need to chain together the results we proved in the previous questions. The K-means algorithm consists of two phases in one full iteration: the assignment phase and the update phase. We can analyze the change in the loss function for each phase separately to see the total effect.

First, from Question 3, we analyzed the assignment step where we fix the centroids and move data points to their closest cluster. We proved that this operation never increase the total distance because we explicitly choose the minimum distance for every point. Therefore, the loss function after reassignment $J_t'$ is less than or equal to the initial loss $J_t$:

$$J_t' \leq J_t$$

Second, from Question 5 and Question 6, we looked at the update step where we fix the assignments and calculate new centroids. We showed that taking the average of the points is the optimal way to minimize the intra-cluster variance. Because the new centroid $\mu^{t+1}$ is the mathematical minimum, it gives a lower or equal error compared to the old centroid. This give us the second inequality:

$$J_{t+1} \leq J_t'$$

Now, we can combine these two inequalities into a single chain. Since $J_{t+1}$ is smaller than $J_t'$, and $J_t'$ is smaller than $J_t$, it follows by transitivity that:

$$J_{t+1} \leq J_t' \leq J_t$$

This result implies that the total error $J_{t+1}$ is always less than or equal to $J_t$. This proves the monotonicity of the K-means algorithm, guaranteeing that the objective function will decrease or stay the same after every complete loop, and the model will not diverge.

#### Question 8:

After recalling a trivial lower bound for the sequence $(J_t)_{t \geq 0}$, what can you say about the convergence?

### **Q8 Ans**
To analyze the convergence, we first identify a trivial lower bound for the loss function. The function $J$ is defined as the sum of squared Euclidean norms:

$$J(\mu_1, ..., \mu_K) = \sum_{i=1}^{n} \lVert x_i - \mu(i)\rVert^2$$

Since the Euclidean norm $\lVert \cdot \rVert$ represents a distance, it is always non-negative. Furthermore, the square of a real number is always non-negative. Therefore, we are summing $n$ non-negative terms, which implies that the total sum cannot be negative. Thus, the sequence of loss values $(J_t)_{t \geq 0}$ is bounded from below by 0:

$$J_t \geq 0 \quad \text{for all } t$$

Next, we recall the result from Question 7, where we proved that the algorithm minimizes the objective function at each step. The assignment step minimizes $J$ with respect to the cluster assignments $c$, and the update step minimizes $J$ with respect to the centroids $\mu$. This process is known as coordinate descent. Because each step reduces (or maintains) the error, the sequence is monotonically decreasing:

$$J_{t+1} \leq J_t$$

Finally, we can apply the Monotone Convergence Theorem from real analysis. This theorem states that any sequence of real numbers that is monotonically decreasing and bounded from below must converge to a limit. Since our sequence $(J_t)$ is always decreasing but can never go below 0, it must converge to a finite value $J^*$.

$$\lim_{t \to \infty} J_t = J^*$$

This proves that the K-means algorithm will always stabilize in terms of the loss function. While it is theoretically possible for the assignments to oscillate between states with the exact same energy, the value of the loss function itself is guaranteed to converge.

#### Question 9:

We just proved that the algorithm converges, but what about its stability:

Let us suppose that the data are sampled from a mixture of $K$ Gaussian, where the choice of $K$ is free for this question. Do you imagine a situation where the algorithm does not classify the data at all? Please design and explain the situation as clearly as possible.

### **Q9 Ans**

While we have proved that the algorithm converges, it does not guarantee finding the best possible solution. This is because the distortion function $J$ is a non-convex function. The K-means algorithm performs coordinate descent on $J$, which means it steps down the "valley" of the loss function, but it can easily get stuck in a local optimum instead of the global minimum. To illustrate a situation where the algorithm fails to classify the data correctly, we can design a scenario based on symmetry.

Let us suppose we have a dataset sampled from a mixture of $K=2$ Gaussian clusters that are well-separated along the x-axis. We can define the true centers of these clusters at:

$$\text{True Center } A = (-10, 0)$$
$$\text{True Center } B = (10, 0)$$

Ideally, the algorithm should classify the data into a "Left Cluster" and a "Right Cluster." However, if we choose the initial centroids poorly, the algorithm might fail. Let us initialize the two centroids on the vertical axis, which acts as the perpendicular bisector between the two data clouds:

$$\mu_1 = (0, 5)$$
$$\mu_2 = (0, -5)$$

In the assignment step, the algorithm looks for the closest centroid for each point. For any data point located in the top half of the plane (positive y), the distance to the top centroid $\mu_1$ is smaller than the distance to the bottom centroid $\mu_2$, regardless of whether the point belongs to the left or right cluster. Consequently, $\mu_1$ captures the top half of Cluster A and the top half of Cluster B. Similarly, $\mu_2$ captures the bottom halves of both clusters.

In the update step, the new centroids are calculated as the average of their assigned points. Because the data is symmetric around the y-axis, the x-coordinates of the left and right points cancel each other out during the averaging process.

$$\mu_{1, x}^{new} \approx \frac{-10 + 10}{2} = 0$$

This means the centroids will move up and down along the y-axis but will never move left or right to find the true clusters. The algorithm will converge to a stable state where it separates the data into "Top" and "Bottom" groups, which is completely orthogonal to the true "Left" and "Right" structure. This situation confirms that K-means is susceptible to local optima and that the initialization is crucial for its success.

#### Question 10:

What can you say about those configurations of centroids? What does it imply concerning the minima? Conclude your arguments by discussing the convexity of the problem. 

### **Q10 Ans**

The configurations of centroids we saw in Question 9 imply that the K-means algorithm is sensitive to initialization. Depending on where we start, the algorithm can converge to different final states. This implies that the objective function $J$ does not have a single, unique valley that leads to the best solution. Instead, the landscape of the loss function is rugged, containing many different valleys of varying depths known as local minima. While the coordinate descent approach guarantees that the loss will decrease and converge, it does not guarantee that the valley we fall into is the deepest one, which would be the global minimum.

This existence of multiple distinct stable states allows us to conclude that the K-means optimization problem is non-convex. In mathematical optimization, a strictly convex function resembles a smooth bowl where any ball dropped inside will roll to the exact same bottom point. Because K-means allows for suboptimal stable states where the algorithm gets stuck, the function cannot be convex. The non-convexity arises largely because of the "min" operator in the assignment step; when a centroid moves just far enough, a data point suddenly switches its assignment from one cluster to another, creating "kinks" or breaks in the function's curvature.

To prove formally that $J$ is not convex, we can use the definition of convexity. A function is convex if, for any two points in the solution space, the function value at the midpoint is lower than or equal to the average of the function values at the two endpoints. If we find one case where the midpoint has a higher error, the function is not convex. Let us consider a simple 1D dataset with two points $x_1 = -1$ and $x_2 = 1$, and we want to find $K=2$ centroids.

First, consider Configuration A, which is an optimal solution:
$$\mu_A = \{-1, 1\}$$
$$J(\mu_A) = (-1 - (-1))^2 + (1 - 1)^2 = 0$$

Next, consider Configuration B, which is the same centroids but swapped (also optimal):
$$\mu_B = \{1, -1\}$$
$$J(\mu_B) = (-1 - (-1))^2 + (1 - 1)^2 = 0$$
(Note: In Configuration B, $x_1$ assigns to the second centroid and $x_2$ assigns to the first, so the distance is still zero).

Now, let us calculate the loss at the geometric midpoint of these two configurations:
$$\mu_{mid} = \frac{\mu_A + \mu_B}{2} = \left\{ \frac{-1+1}{2}, \frac{1+(-1)}{2} \right\} = \{0, 0\}$$

In this midpoint configuration, both centroids are at 0. Both data points $x_1$ and $x_2$ are distance 1 away from the origin:
$$J(\mu_{mid}) = (-1 - 0)^2 + (1 - 0)^2 = 1 + 1 = 2$$

Finally, we compare the midpoint loss to the average loss of the endpoints:
$$J(\mu_{mid}) = 2$$
$$\text{Average}(J(\mu_A), J(\mu_B)) = \frac{0 + 0}{2} = 0$$

Since $2 > 0$, the inequality required for convexity is violated. The "bowl" curves upward in the middle, proving the function is non-convex. Because of this property, a common practical solution is to run K-means multiple times with different random initializations and pick the result with the lowest distortion $J$.

#### Question 11:

We can also quickly generalize our algorithm.

In some situation, you are aiming at favoring some directions in your data and penalizing the others, so that you can weigh the euclidean distance according to:

$$d^{(w)}(x_{i}, \mu(i)) = \frac{\sum_{j=1}^d w_i(x_{ij} - \mu(i)_j)^2}{\sum_{j=1}^d w_j} $$

Show that with a change of variables, the problem remains the same.


### **Q11 Ans**

To solve Question 11, we need to show that introducing directional weights into the distance metric is mathematically equivalent to solving the standard K-means problem on a rescaled version of the dataset.

First, let us address the notation in the problem statement. The formula provided in the question uses $w_i$ inside a sum over dimensions $j$, but the denominator sums $w_j$, and the context mentions "favoring some directions." This strongly suggests a typo where the weight should depend on the dimension $j$, not the data point index $i$. We will assume the correct term is $w_j$.

We start by writing down the total loss function using the weighted distance provided. Let $W_{total} = \sum_{j=1}^d w_j$ be the normalization constant. The weighted objective function $J_{weighted}$ sums this distance over all points $i=1$ to $n$:

$$J_{weighted} = \sum_{i=1}^n d^{(w)}(x_{i}, \mu(i)) = \sum_{i=1}^n \frac{1}{W_{total}} \sum_{j=1}^d w_j (x_{ij} - \mu(i)_j)^2$$

Since $W_{total}$ is just a constant positive scalar, minimizing $J_{weighted}$ is equivalent to minimizing the numerator alone. Let us focus on the term inside the summation:

$$J_{numerator} = \sum_{i=1}^n \sum_{j=1}^d w_j (x_{ij} - \mu(i)_j)^2$$

We want to transform this equation into the standard squared Euclidean form $\sum (a - b)^2$. We can do this by absorbing the weight $w_j$ into the squared term. Recall that $c \cdot z^2 = (\sqrt{c} \cdot z)^2$ for any non-negative weight $c$.

We define a change of variables to create a new dataset $\tilde{x}$ and new centroids $\tilde{\mu}$:

$$\tilde{x}_{ij} = \sqrt{w_j} \cdot x_{ij}$$

$$\tilde{\mu}(i)_j = \sqrt{w_j} \cdot \mu(i)_j$$

This transformation effectively stretches or shrinks the data along each dimension $j$ based on how important that direction is (the square root of the weight $w_j$). Now, we substitute these new variables back into our objective function:

$$J_{numerator} = \sum_{i=1}^n \sum_{j=1}^d \left( \sqrt{w_j} (x_{ij} - \mu(i)_j) \right)^2$$

$$= \sum_{i=1}^n \sum_{j=1}^d \left( \sqrt{w_j} x_{ij} - \sqrt{w_j} \mu(i)_j \right)^2$$

$$= \sum_{i=1}^n \sum_{j=1}^d \left( \tilde{x}_{ij} - \tilde{\mu}(i)_j \right)^2$$

We can recognize that the inner sum $\sum_{j=1}^d (\dots)^2$ is simply the standard squared Euclidean norm for the transformed vectors. Therefore, the weighted problem can be rewritten as:

$$J_{weighted} \propto \sum_{i=1}^n \lVert \tilde{x}_i - \tilde{\mu}(i) \rVert^2$$

This result shows that the objective function is exactly the same as the standard K-means loss function, but applied to the transformed data $\tilde{x}$. The problem remains the same structurally. Practically, this means you do not need to implement a special "weighted K-means" algorithm. You can simply multiply each column of your data matrix by $\sqrt{w_j}$, run the standard K-means algorithm, and then divide the resulting centroids by $\sqrt{w_j}$ to get the solution in the original space.

# Part 2: Restricted Boltzmann Machine




### Introduction

The Boltzmann Machine have been inspired by thermodynamic and statistical physics models, more precisely they are part of the Energy Models using the well known Boltzmann Distribution as written in physics style:

$$ P\left( E \right)  = \frac{1}{Z} \exp \left( -\frac{E}{k_b T} \right)$$

It becomes in statistical inference framework:
$$
P(\mathbf{v} | J, \mathbf{b}) \propto e^{\mathbf{v}^TJ\mathbf{v} + \mathbf{b}^T\mathbf{v}} = e^{-E(\mathbf{v})}
$$
where:
- $\mathbf{v}\in\mathbb{R}^n:$ The binary vector with components $v_i = 0 \; {\rm or} \; 1$

- $J \in \mathbb{R}^{n \times n}:$ The coupling matrix

- $\mathbf{b} \in  \mathbb{R}^n$: Field

- $E(\mathbf{v}) \in  \mathbb{R}$: Energy


However, one problem arised with initial Boltzmann Machine (BM) -- like its parent models in statistical physics (as the SK model) -- all the units are interacting through complicated dependencies. For example, if we consider 3 components of $\mathbf{v}$: $v_1$, $v_2$, and $v_3$, there are trivial interactions such as one modelised by $P(v_1, v_2)$ corresponding to the correlation between the two first components of $\mathbf{v}$, but there are also none trivial interactions. Indeed, if some term like $P(v_1, v_2 | v_3)$ which suggests that the correlation $x_1$ and $v_2$ depends on $v_3$ and this is clearly none linear.

A really ingenious way to overcome this situation is to replace all the tricky interactions between the units $\mathbf{v}\in\mathbb{R}^n$ by connections through hidden units $\mathbf{h}\in\mathbb{R}^m$, artifically created. Indeed, correlations between two units $v_1$ and $v_2$ (specially the dependency of their correlations on other units $v_3$, $v_4$,...) can be atrificially replaced by introducing a third unit $h_1$ and considerin only linear correlations between $v_1 \leftrightarrow h_1$, $h_1 \leftrightarrow v_2$ and $v_1 \leftrightarrow v_2$. The units $v_i$ are now called the visible units. This model is the most known version of BMs. 

<div style="text-align: center;">
    <img src="boltzmannmachine.png" alt="Diagram here" />
</div>


However, this model is still fully connected and makes the computation really costful. Then, one can even simplify the model by considering zero intra layer interractions. This simplified model is call Restricted Boltzmann Machine (RBM) (Physics Nobel Price 2024 🥳).

Thus, the RBM architecture consists of two layers of binary stochastic units: a $\textbf{visible layer}$ $\mathbf{v}$ and a $\textbf{hidden layer}$ $\mathbf{h}$. The layers are fully connected, but there are no connections within a layer, making the model a $\textbf{bipartite graph}$. 

<div style="text-align: center;">
    <img src="rbm.png" alt="Diagram here" />
</div>

Restricted Boltzmann Machines (RBMs) are a class of energy-based probabilistic graphical models that are commonly used in machine learning for tasks such as dimensionality reduction, feature learning, and generative modeling.

### Energy Function and Probabilities

The joint configuration of the visible units $\mathbf{v} \in \{0, 1\}^d$ and the hidden units $\mathbf{h} \in \{0, 1\}^m$ is associated with an $\textbf{energy function}$, defined as:

$$ E(\mathbf{v}, \mathbf{h}) = -\mathbf{v}^\top \mathbf{W} \mathbf{h} - \mathbf{b}^\top \mathbf{v} - \mathbf{c}^\top \mathbf{h}$$
where:
- $\mathbf{W} \in \mathbb{R}^{d \times m}$ is the weight matrix connecting the visible and hidden units,
- $\mathbf{b} \in \mathbb{R}^d$ field of the visible units or also called the biases of the visible units,
- $\mathbf{c} \in \mathbb{R}^m$ field of the hidden units of also called the biases of the hidden units.

The energy function determines the joint probability distribution over $\mathbf{v}$ and $\mathbf{h}$:
$$ P(\mathbf{v}, \mathbf{h}) = \frac{1}{Z} \exp(-E(\mathbf{v}, \mathbf{h})) $$
where $Z$ is the $\textbf{partition function}$, ensuring normalization:

$$ Z = \sum_{\mathbf{v}, \mathbf{h}} \exp(-E(\mathbf{v}, \mathbf{h})) $$


The marginal probability of the visible units $\mathbf{v}$ is obtained by summing over all possible configurations of the hidden units:

$$ P(\mathbf{v}) = \frac{1}{Z} \sum_{\mathbf{h}} \exp(-E(\mathbf{v}, \mathbf{h})). $$

Based on the material you provided, here is an explanation of the **origin** of the Boltzmann Machine and the specific **problems** it was designed to solve.

根据你提供的材料，以下是对玻尔兹曼机器的**起源**及其旨在解决的具体**问题**的解释。

---

### 1. Where does it come from? (Origin)
### 1. 它从哪里来？（起源）

**Physics Inspiration**
**物理学灵感**

* **English:** According to the text, the Boltzmann Machine is directly inspired by **thermodynamic and statistical physics models**. It belongs to a class called "Energy Models."
* **Chinese:** 根据文本，玻尔兹曼机直接受**热力学和统计物理模型**的启发。它属于“能量模型”这一类。

* **English:** Its mathematical foundation comes from the **Boltzmann Distribution** used in physics to describe the probability of a state based on its energy:
    $$P(E) = \frac{1}{Z} \exp\left(-\frac{E}{k_b T}\right)$$
    The machine learning version adapts this formula to calculate the probability of data vectors ($\mathbf{v}$) based on an Energy function ($E$).
* **Chinese:** 它的数学基础来自物理学中使用的**玻尔兹曼分布**，用于根据能量描述某种状态的概率。机器学习版本采用了这个公式，根据能量函数 ($E$) 来计算数据向量 ($\mathbf{v}$) 的概率。

---

### 2. What problem does it solve?
### 2. 它解决了什么问题？

The material describes a two-step evolution to solve two specific problems: **Complexity** and **Computational Cost**.
材料描述了一个两步演进过程，旨在解决两个具体问题：**复杂性**和**计算成本**。

#### Problem A: Modeling Complex Dependencies
#### 问题 A：对复杂的依赖关系建模

* **The Issue (Original Concept):** In early models, visible units had complicated, non-linear interactions (e.g., the correlation between $v_1$ and $v_2$ might depend on $v_3$). This is hard to model directly.
* **问题（原始概念）：** 在早期的模型中，可见单元之间存在复杂的非线性相互作用（例如，$v_1$ 和 $v_2$ 之间的相关性可能取决于 $v_3$）。这很难直接建模。

* **The Solution (Hidden Units):** The material describes an "ingenious way" to fix this: introducing **Hidden Nodes** ($\mathbf{h}$). Instead of $v_1$ interacting directly with $v_2$ in complex ways, they interact through a hidden unit ($h_1$). This allows the model to capture complex patterns using simpler linear connections.
* **解决方案（隐藏单元）：** 材料描述了一个“巧妙的方法”来解决这个问题：引入**隐藏节点** ($\mathbf{h}$)。$v_1$ 不再直接以复杂的方式与 $v_2$ 交互，而是通过隐藏单元 ($h_1$) 进行交互。这使得模型能够使用更简单的线性连接来捕捉复杂的模式。

#### Problem B: Computational Cost (The birth of RBM)
#### 问题 B：计算成本（RBM 的诞生）

* **The Issue (Standard Boltzmann Machine):** Even with hidden units, the standard Boltzmann Machine is **fully connected** (every node connects to every other node). The text states this "makes the computation really costful."
* **问题（标准玻尔兹曼机）：** 即使有了隐藏单元，标准的玻尔兹曼机也是**全连接的**（每个节点都与所有其他节点相连）。文本指出这“使得计算成本非常高昂”。

* **The Solution (Restricted Boltzmann Machine - RBM):** To solve this, the connections are **restricted**.
    * There are **zero intra-layer interactions** (no connections between visible nodes, no connections between hidden nodes).
    * This creates a **Bipartite Graph** structure (as seen in your second image).
* **解决方案（受限玻尔兹曼机 - RBM）：** 为了解决这个问题，连接被**限制**了。
    * **层内交互为零**（可见节点之间没有连接，隐藏节点之间也没有连接）。
    * 这这就产生了**二分图**结构（如你的第二张图所示）。

### Summary
### 总结

* **Origin:** Physics (Thermodynamics).
* **Problem:** Modeling complex data relationships was too hard, and the calculations were too slow.
* **Solution:** Use **Hidden Units** to simplify the relationships, and use the **Restricted** architecture (no layer-internal connections) to make the math faster and feasible.

* **起源：** 物理学（热力学）。
* **问题：** 对复杂的数据关系建模太难，且计算速度太慢。
* **解决方案：** 使用**隐藏单元**来简化关系，并使用**受限**架构（无层内连接）使数学计算更快、更可行。

---

**Next Step:**
The text mentions that RBMs create a **"Bipartite Graph"** (二分图). Would you like me to explain why this specific structure makes the math so much easier compared to the full Boltzmann Machine?
文本中提到 RBM 建立了一个**“二分图”**。你想让我解释一下，为什么这种特定的结构比全连接的玻尔兹曼机能让数学计算变得如此简单吗？

这是受限玻尔兹曼机（RBM）理论中最核心的公式之一。它描述的是：**如果我们忽略隐藏层发生的一切，仅仅观察到某一个特定数据样本（比如一张图片 $\mathbf{v}$）的概率是多少。**

我们可以从三个层面来理解这个公式：**直觉含义**、**数学操作**和**实际作用**。

### 1. 直觉含义：所有可能性的“总和”
### Intuitive Meaning: The "Sum" of All Possibilities

* **Joint vs. Marginal (联合 vs. 边缘):**
    * $P(\mathbf{v}, \mathbf{h})$ 是联合概率。它描述的是：“看见图片 $\mathbf{v}$ **且** 隐藏特征是 $\mathbf{h}$” 的概率。
    * $P(\mathbf{v})$ 是边缘概率。它描述的是：“看见图片 $\mathbf{v}$” 的概率，**不管 $\mathbf{h}$ 是什么**。

* **Analogy (比喻):**
    想象 $\mathbf{v}$ 是你在森林里看到的一个“脚印”。
    $\mathbf{h}$ 是可能留下这个脚印的动物（熊、老虎、或者穿着大鞋子的人）。
    
    要计算**这个脚印出现的概率 $P(\mathbf{v})$**，你不能只考虑“熊”留下的概率，你必须把“熊留下的概率” + “老虎留下的概率” + “人留下的概率”... **全部加起来**。
    
    这就是公式中 $\sum_{\mathbf{h}}$ 的含义：把所有可能产生 $\mathbf{v}$ 的隐藏情况 $\mathbf{h}$ 的贡献值全部加总。

---

### 2. 数学操作：消除未知变量
### Mathematical Operation: Removing the Unknowns

让我们拆解公式中的每一部分：

$$P(\mathbf{v}) = \frac{1}{Z} \sum_{\mathbf{h}} \exp(-E(\mathbf{v}, \mathbf{h}))$$

1.  **$E(\mathbf{v}, \mathbf{h})$ (能量 Energy):**
    这是一个衡量 $\mathbf{v}$ 和 $\mathbf{h}$ “兼容性”的分数。能量越低，它们越匹配。
    * *Lower Energy = Better Match.*

2.  **$\exp(-E(\dots))$ (非归一化概率 Unnormalized Probability):**
    这是一个正数。能量越低，这个值越大。它代表某种特定的隐藏状态 $\mathbf{h}$ 对可见状态 $\mathbf{v}$ 的“支持程度”。

3.  **$\sum_{\mathbf{h}}$ (对 $\mathbf{h}$ 求和 Sum over h):**
    因为 $\mathbf{h}$ 是隐藏的（我们在现实中看不到它），它是**不确定**的。
    为了得到 $\mathbf{v}$ 的真实概率，我们必须遍历 $\mathbf{h}$ 的所有可能取值（比如 $h=\{0,0\}, \{0,1\}, \{1,0\}, \{1,1\}$），算出它们每一个与 $\mathbf{v}$ 的匹配度，然后加在一起。
    * 这个过程在数学上叫 **Marginalization (边缘化)**。

4.  **$\frac{1}{Z}$ (归一化 Normalization):**
    这只是为了确保最后算出来的结果是一个 0 到 1 之间的概率值。

---

### 3. 实际作用：模型训练的目标
### Practical Use: The Goal of Training

你可能会问：“为什么我们要算这个？”

因为在训练 RBM 时，我们只有数据 $\mathbf{v}$（比如一堆手写数字的图片），没有 $\mathbf{h}$。
我们的目标是调整权重 $\mathbf{W}$，使得模型生成我们训练数据的概率 $P(\mathbf{v})$ **最大化**。

* 如果我们给模型看一张真实的狗的图片 $\mathbf{v}_{dog}$，我们希望 $P(\mathbf{v}_{dog})$ 很高。
* 这意味着我们需要调整参数，使得**总能量**（在所有可能的 $\mathbf{h}$ 组合下）尽可能低。

### 总结 (Summary)

这个公式是在说：
> **“为了算出这个数据 $\mathbf{v}$ 出现的可能性，请把所有能通过隐藏层 $\mathbf{h}$ 解释出 $\mathbf{v}$ 的路径都加起来。”**

---

**下一步 (Next Step):**
虽然这个求和符号 $\sum_{\mathbf{h}}$ 看起来很可怕（因为 $\mathbf{h}$ 的组合可能有指数级那么多），但在 RBM 中，由于层内没有连接，我们可以用一个非常漂亮的数学技巧把这个连加变成简单的连乘（这就引出了 **Free Energy 自由能** 的概念）。

**你想看它是如何通过“自由能”公式被简化计算的吗？**

#### Question 12:

Write a valid expression of the energy $E(\textbf{v}, \textbf{h})$ in the case of a BM (non-restricted) with an hidden layer. 

### **Q12 Ans**

To write the valid expression for the energy function in a general Boltzmann machine, we first need to decompose the total units x into two subsets: the visible units v and the latent or hidden units h. Unlike the Restricted Boltzmann Machine where layers are independent, the general Boltzmann machine allows connections between all variables, meaning we must account for interactions within the visible layer and within the hidden layer as well.

Following the mathematical convention presented by Goodfellow et al. (2016, Eq. 20.3), the valid expression for the energy function E(v, h) is defined as:

$$E(\mathbf{v}, \mathbf{h}) = -\mathbf{v}^\top \mathbf{R} \mathbf{v} - \mathbf{v}^\top \mathbf{W} \mathbf{h} - \mathbf{h}^\top \mathbf{S} \mathbf{h} - \mathbf{b}^\top \mathbf{v} - \mathbf{c}^\top \mathbf{h}$$

In this expression, the matrix W represents the weights connecting the visible and hidden units, which is the standard interaction term. However, because this is a non-restricted machine, we also include the matrix R and matrix S. The matrix R captures the pairwise interactions between the visible units themselves, and S capture the interactions among the hidden units. Finally, the vectors b and c corresponds to the bias parameters for the visible and hidden layers respectively. This full connectivity allow the model to represent higher-order correlations that simpler models cannot.

### Conditional Independence

#### Question 13:

One of the key properties of RBMs is the $\textbf{conditional independence}$ between units within a layer:

Compute the conditional probability and show that:

$$ P(h_j = 1 | \mathbf{v}) = \sigma\left(c_j + \sum_{i} v_i W_{ij}\right) $$
and
$$ P(v_i = 1 | \mathbf{h}) = \sigma\left(b_i + \sum_{j} h_j W_{ij}\right) $$

where $\sigma(x) = \frac{1}{1 + e^{-x}}$ is the sigmoid activation function.

This bipartite structure enables efficient Gibbs sampling for approximating the intractable joint distribution $P(\mathbf{v}, \mathbf{h})$.

### **Q13 Ans**

### Learning in RBMs

#### Question 14:

Training an RBM involves maximizing the likelihood of the data distribution. To do so we are aiming at using a gradient descent/ascent on the weights (and biases).

Compute the log-likelihood $\mathcal{L}(\mathbf{v})$, remember that the model is part of the unsupervised learning.

### **Q14 Ans**

#### Question 15:

Compute the gradient of the log-likelihood with respect to the weights $\mathbf{W}$ and the biases $\mathbf{b}$, $\mathbf{c}$ : 

### **Q15 Ans**

Now, it should be possible to implement the RBM!

#### Question 16: (Open question)

While it seems possible to run RBM algorithm, note that the second term in the gradient w.r.t. $\mathbf{W}$ is computationally expensive due to the intractability of $Z$, the approximation Contrastive Divergence - k is often use. Research what is this approximation, is this approximation enough, why? Explain it with your own words and cite the papers you used for your documentation.

### **Q16 Ans**

### Applications of RBMs

RBMs are widely used in tasks such as:

- $\textbf{Dimensionality reduction}$: Similar to PCA but capable of capturing non-linear structures,
- $\textbf{Feature learning}$: For pre-training deep neural networks,
- $\textbf{Collaborative filtering}$: Used in recommendation systems.

# Reference

[1] Goodfellow, I., Bengio, Y., & Courville, A. (2016). *Deep learning*. MIT Press.